In [2]:
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import Qubit,QubitBra
init_printing() # to show vectors and matrices nicely
from sympy.physics.quantum.gate import X,Y,Z,H,S,T,CNOT,SWAP,CPHASE,CGateS
import numpy as np

QFT

To deepen our understanding of the quantum Fourier transform, let’s implement the circuit for n = 3 using SymPy.

First, the input to Fourier transform, $\ket{x}$, is $\ket{x} = \sum_{j=0}^7 \frac{1}{\sqrt{8}} \ket{j}$

Fourier transform of the array corresponding to this state in a classical way is:

In [3]:

input_np_array = 1/np.sqrt(8)*np.ones(8)
print( input_np_array ) ## input
print(np.fft.ifft(input_np_array) * np.sqrt(8) )
#To match the definition of Fourier transform with the definition of ifft in numpy, multiply by sqrt(2^3)

[0.35355339 0.35355339 0.35355339 0.35355339 0.35355339 0.35355339
 0.35355339 0.35355339]
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


Let us verify this by quantum Fourier transform.

In [4]:
input = 1/sqrt(8) *( Qubit("000")+Qubit("001")+Qubit("010")+Qubit("011")+Qubit("100")+Qubit("101")+Qubit("110")+Qubit("111"))
input

√2⋅(❘000⟩ + ❘001⟩ + ❘010⟩ + ❘011⟩ + ❘100⟩ + ❘101⟩ + ❘110⟩ + ❘111⟩)
──────────────────────────────────────────────────────────────────
                                4                                 

Let's define $R_1$ $R_2$ e $R_3$ from the library with Z,S,T, respectively $e^{i\pi} = -1$, $e^{i\pi/2} = i$ e $e^{i\pi/4}$

In [5]:
represent(Z(0),nqubits=1), represent(S(0),nqubits=1), represent(T(0),nqubits=1)

⎛                 ⎡1   0  ⎤⎞
⎜                 ⎢       ⎥⎟
⎜⎡1  0 ⎤  ⎡1  0⎤  ⎢    ⅈ⋅π⎥⎟
⎜⎢     ⎥, ⎢    ⎥, ⎢    ───⎥⎟
⎜⎣0  -1⎦  ⎣0  ⅈ⎦  ⎢     4 ⎥⎟
⎝                 ⎣0  ℯ   ⎦⎠

We will construct a circuit that performs a quantum Fourier transform (we abbreviate Quantum Fourier Transform as QFT).
First, we apply the Hadamard operator to the first (second in SymPy, since SymPy counts bits 0, 1, 2 from the right) qubit, and then apply $R_2$ e $R_3$ gates with the second and third bits as control bits.

In [6]:
QFT_gate = H(2)
QFT_gate = CGateS(1, S(2))  * QFT_gate
QFT_gate = CGateS(0, T(2))  * QFT_gate

The second (first in SymPy) qubit is also subjected to the Hadamard gate $R_2$ and control operation.

In [7]:
QFT_gate = H(1)  * QFT_gate
QFT_gate = CGateS(0, S(1))  * QFT_gate

The third (0th in SymPy) qubit should only have an Hadamard gate applied.

In [8]:
QFT_gate = H(0)  * QFT_gate

Finally, a SWAP gate is applied to match the order of the bits.

In [10]:
QFT_gate = SWAP(0, 2)  * QFT_gate
QFT_gate

         2                              
⎛SWAP   ⎞ ⋅H ⋅C ⎛S ⎞⋅H ⋅C ⎛T ⎞⋅C ⎛S ⎞⋅H 
⎝    0,2⎠   0  0⎝ 1⎠  1  0⎝ 2⎠  1⎝ 2⎠  2

In [11]:
simplify( qapply( QFT_gate * input) )

❘000⟩

Phase Estimation Algorithm

$4 \times 4$ matrix U is constructed using T and S operations, and the eigenvalues of this matrix are calculated

In [12]:
represent(T(0)*S(1),nqubits = 2)

⎡1   0    0    0   ⎤
⎢                  ⎥
⎢    ⅈ⋅π           ⎥
⎢    ───           ⎥
⎢     4            ⎥
⎢0  ℯ     0    0   ⎥
⎢                  ⎥
⎢0   0    ⅈ    0   ⎥
⎢                  ⎥
⎢               ⅈ⋅π⎥
⎢               ───⎥
⎢                4 ⎥
⎣0   0    0  ⅈ⋅ℯ   ⎦

This matrix is already diagonalized, and the eigenvalue phases $\lambda$ are 0, $\pi/4$, $\pi/2$, $3pi/4$, whose binary decimal representations are
($2\pi$)0.0, ($2\pi$)0.001, ($2\pi$)0.011. Therefore, three auxiliary qubits are needed to measure these without error.
Using the CGateS function, let us define controlled U-gate cP_2,3,4. (2,3,4) correspond to the three auxiliary qubits. (0,1) are the spaces in which U acts. In the previous schematic of phase estimation, the bits are named 0,1,2,... from the bottom.

In [13]:
cP_2 = CGateS(2,T(0))*CGateS(2,S(1))
cP_3 = CGateS(3,T(0))*CGateS(3,S(1))
cP_4 = CGateS(4,T(0))*CGateS(4,S(1))

Let us construct a phase estimation algorithm using three auxiliary qubits.
First, Hadamard gate is applied to all auxiliary qubits.

In [14]:
PhaEst = H(4)*H(3)*H(2)
PhaEst

H ⋅H ⋅H 
 4  3  2

Next, apply the control unitary once, twice, and four times, respectively.

In [15]:
PhaEst = cP_2*cP_3*cP_3*cP_4*cP_4*cP_4*cP_4*PhaEst

This completes the first half of the phase estimation algorithm. From here, the inverse quantum Fourier transform is constructed.
First, for the third decimal place, we apply the H operation to the auxiliary qubit 4.

In [16]:
PhaEst = H(4)*PhaEst

The auxiliary qubit 3 is first subjected to the control $R_2^+ = S^{-1} = SZ$ operation with auxiliary qubit 4 as the control.

In [17]:
PhaEst = CGateS(4,S(3))*PhaEst
PhaEst = CGateS(4,Z(3))*PhaEst

The H operation is then applied.

In [18]:
PhaEst = H(3)*PhaEst

The three operations are applied to auxiliary qubit 2
$R_2^+ = S^{-1} = SZ$ operation with auxiliary qubit 3 as control qubit
$R_3^+ = TS^{-1} = TSZ$
H operation

In [19]:
PhaEst = CGateS(3,S(2))*PhaEst
PhaEst = CGateS(3,Z(2))*PhaEst

PhaEst = CGateS(4,T(2))*PhaEst
PhaEst = CGateS(4,S(2))*PhaEst
PhaEst = CGateS(4,Z(2))*PhaEst
PhaEst = H(2)*PhaEst

Let’s apply the phase estimation algorithm constructed in this way to the eigenvectors. The algorithm itself is very complex.

In [20]:
PhaEst

H ⋅C ⎛Z ⎞⋅C ⎛S ⎞⋅C ⎛T ⎞⋅C ⎛Z ⎞⋅C ⎛S ⎞⋅H ⋅C ⎛Z ⎞⋅C ⎛S ⎞⋅H ⋅C ⎛T ⎞⋅C ⎛S ⎞⋅C ⎛T ⎞
 2  4⎝ 2⎠  4⎝ 2⎠  4⎝ 2⎠  3⎝ 2⎠  3⎝ 2⎠  3  4⎝ 3⎠  4⎝ 3⎠  4  2⎝ 0⎠  2⎝ 1⎠  3⎝ 0⎠

⋅C ⎛S ⎞⋅C ⎛T ⎞⋅C ⎛S ⎞⋅C ⎛T ⎞⋅C ⎛S ⎞⋅C ⎛T ⎞⋅C ⎛S ⎞⋅C ⎛T ⎞⋅C ⎛S ⎞⋅C ⎛T ⎞⋅C ⎛S ⎞⋅
  3⎝ 1⎠  3⎝ 0⎠  3⎝ 1⎠  4⎝ 0⎠  4⎝ 1⎠  4⎝ 0⎠  4⎝ 1⎠  4⎝ 0⎠  4⎝ 1⎠  4⎝ 0⎠  4⎝ 1⎠ 

H ⋅H ⋅H 
 4  3  2

However, when the algorithm act on the input,

In [21]:
simplify(qapply(PhaEst*Qubit("00011")))

❘11011⟩

The output is simple as designed. The input $\ket{\phi} = \ket{11}$, and the corresponding eigenvalue is $e^{i3\pi /4}$. The auxiliary qubits 2,3,4 are 011, yielding the binary fraction representation of eigenvalue phase $\lambda = 3\pi /4$. 
For other inputs:

In [22]:
simplify(qapply(PhaEst*Qubit("00000")))

❘00000⟩

In [23]:
simplify(qapply(PhaEst*Qubit("00010")))

❘01010⟩

In [24]:
simplify(qapply(PhaEst*Qubit("00001")))

❘10001⟩

and it can be verified that the phase information of the four eigenvalues is obtained in the three auxiliary qubits. Also, if the input state is a superposition state,

In [25]:
simplify(qapply(PhaEst*H(0)*H(1)*Qubit("00000")))

❘00000⟩ + ❘01010⟩ + ❘10001⟩ + ❘11011⟩
─────────────────────────────────────
                  2                  

For each eigenvector, the phase of the eigenvalue is taken out into three auxiliary qubits as a superposition. Measuring the auxiliary qubits in this state yields any one eigenvector and eigenvalue probabilistically.